#Проект работы по прогнозированию результативности игроков NHL

In [67]:
# Подключение библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# Inline plotting
%matplotlib inline
# показать все колонки df
pd.set_option('display.max_columns', None)

In [68]:
# монтирование Google disk
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [69]:
# загрузка файла, содержащего статистику игроков link - https://drive.google.com/file/d/1lqiVkcCoZdlPlM5WOj8jdtbeqQQd7OJH/view?usp=drive_link
stats_df = pd.read_csv('/content/drive/My Drive/BigData_proj/NHL_Players_Statistics.csv', sep=';')


In [70]:
stats_df.head()

,Name,Date_of_birth,SEASON_year,SEASON,TEAM,Games_Played,Goals,Assists,Points,PlusMinus_Ratings,Penalty_Minutes,Shots_on_Goal,Shooting_Percentage,PowerPlay_Goals,PowerPlay_Assists,Short_Goals,Short_Assists,Game_Winning_Goals,Game_Tying_Goals,Time_on_Ice_per_Game,Production,Number,Games_Started,Wins,Losses,Ties,Overtime_Losses,Goals_Against,Goals_Against_Average,Shots_Against,Saves,Save_Percentage,Shutouts,Position,Height,Weight,Body_mass_index,Place_of_birth,Age,Experience
0,Aaron Downey,1974-08-27,2000,'99-'00,BOS,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8:31,0:00,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Right_wing,185,98,28.6,"Shelburne, Ontario",26,1
1,Aaron Downey,1974-08-27,2001,'00-'01,CHI,3,0.0,0.0,0.0,-1.0,6.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5:30,0:00,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Right_wing,185,98,28.6,"Shelburne, Ontario",27,2
2,Aaron Downey,1974-08-27,2002,'01-'02,CHI,36,1.0,0.0,1.0,-2.0,76.0,10.0,10.0,0.0,0.0,0.0,0.0,1.0,0.0,5:06,183:38,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Right_wing,185,98,28.6,"Shelburne, Ontario",28,3
3,Aaron Downey,1974-08-27,2003,'02-'03,DAL,43,1.0,1.0,2.0,1.0,69.0,14.0,7.1,0.0,0.0,0.0,0.0,0.0,0.0,4:47,102:40,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Right_wing,185,98,28.6,"Shelburne, Ontario",29,4
4,Aaron Downey,1974-08-27,2004,'03-'04,DAL,37,1.0,1.0,2.0,2.0,77.0,11.0,9.1,0.0,0.0,0.0,0.0,1.0,0.0,4:30,83:19,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Right_wing,185,98,28.6,"Shelburne, Ontario",30,5


In [71]:
# Убирание NAN И перевод в int
#stats_df = stats_df.fillna(0)
#stats_df[['Goals', 'Assists', 'Points', 'PlusMinus_Ratings', 'Penalty_Minutes',
#         'Shots_on_Goal', 'PowerPlay_Goals', 'PowerPlay_Assists', 'Short_Goals',
#         'Short_Assists', 'Game_Winning_Goals', 'Game_Tying_Goals']] = stats_df[['Goals',
#         'Assists', 'Points', 'PlusMinus_Ratings', 'Penalty_Minutes',
#         'Shots_on_Goal', 'PowerPlay_Goals', 'PowerPlay_Assists', 'Short_Goals',
#         'Short_Assists', 'Game_Winning_Goals', 'Game_Tying_Goals']].astype(int)

In [72]:
np.sort(stats_df.SEASON_year.unique())

array([1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986,
       1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997,
       1998, 1999, 2000, 2001, 2002, 2003, 2004, 2006, 2007, 2008, 2009,
       2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])

## Гипотеза о возможности прогноза кол-ва очков на следующий сезон по показателям текущего сезона

In [73]:
# Убираем фичи, которые предположительно не окажут влияния на точность прогноза

stats_df_second = stats_df.copy()
stats_df_second = stats_df_second.drop(['Date_of_birth', 'SEASON', 'TEAM', 'Shooting_Percentage',
                     'PowerPlay_Goals', 'PowerPlay_Assists', 'Short_Goals', 'Short_Assists',
                     'Game_Winning_Goals', 'Game_Tying_Goals', 'Production', 'Number',
                     'Games_Started', 'Wins', 'Losses', 'Ties', 'Overtime_Losses',
                     'Goals_Against', 'Goals_Against_Average', 'Shots_Against',
                     'Saves', 'Save_Percentage', 'Shutouts', 'Position', 'Height', 'Weight',
                     'Body_mass_index', 'Place_of_birth', 'Age', 'Time_on_Ice_per_Game'], axis=1)

In [74]:
# суммируем значение по столбцам, когда в одном сезоне игрок выступал за разные клубы
stats_df_second_gr = stats_df_second.groupby(['Name', 'SEASON_year']).agg({'Games_Played': ['sum'], 'Goals': ['sum'], 'Assists': ['sum'], 'Points': ['sum'],
                                                      'PlusMinus_Ratings': ['sum'], 'Penalty_Minutes': ['sum'], 'Shots_on_Goal': ['sum'],
                                                      'Experience':['max'] })

In [75]:
# после суммирования преобразуем обратно в нормальный датафрейм

stats_df_second_gr = stats_df_second_gr.droplevel(1, axis=1)
stats_df_second = stats_df_second_gr.reset_index()
stats_df_second.drop('SEASON_year', axis=1, inplace=True)
stats_df_second.head(10)

,Name,Games_Played,Goals,Assists,Points,PlusMinus_Ratings,Penalty_Minutes,Shots_on_Goal,Experience
0,Aaron Downey,1,0.0,0.0,0.0,0.0,0.0,0.0,1
1,Aaron Downey,3,0.0,0.0,0.0,-1.0,6.0,2.0,2
2,Aaron Downey,36,1.0,0.0,1.0,-2.0,76.0,10.0,3
3,Aaron Downey,43,1.0,1.0,2.0,1.0,69.0,14.0,4
4,Aaron Downey,37,1.0,1.0,2.0,2.0,77.0,11.0,5
5,Aaron Downey,42,3.0,4.0,7.0,2.0,95.0,21.0,7
6,Aaron Downey,21,1.0,0.0,1.0,-6.0,48.0,10.0,8
7,Aaron Downey,56,0.0,3.0,3.0,0.0,116.0,15.0,9
8,Aaron Downey,4,1.0,1.0,2.0,0.0,7.0,2.0,10
9,Aaron Gagnon,2,0.0,0.0,0.0,0.0,0.0,2.0,1


In [76]:
# Датафрейм для "сдвига" очков на сезон

stats_df_second_point = stats_df_second[['Name', 'Experience', 'Points']].copy()
stats_df_second_point

,Name,Experience,Points
0,Aaron Downey,1,0.0
1,Aaron Downey,2,0.0
2,Aaron Downey,3,1.0
3,Aaron Downey,4,2.0
4,Aaron Downey,5,2.0
...,...,...,...
24812,Zigmund Palffy,8,89.0
24813,Zigmund Palffy,9,59.0
24814,Zigmund Palffy,10,85.0
24815,Zigmund Palffy,11,41.0


In [77]:
# Собственно сам сдвиг за счет пересчета столбца "Опыт в NHL" - 'Experience'

stats_df_second_point['new_exper'] = stats_df_second_point['Experience'] -1
stats_df_second_point.rename(columns = {'Points':'NS_points'}, inplace = True )
stats_df_second_point.drop('Experience', axis=1, inplace=True)
stats_df_second_point.rename(columns = {'new_exper':'Experience'}, inplace = True )
stats_df_second_point.head()

,Name,NS_points,Experience
0,Aaron Downey,0.0,0
1,Aaron Downey,0.0,1
2,Aaron Downey,1.0,2
3,Aaron Downey,2.0,3
4,Aaron Downey,2.0,4


In [78]:
# Слияние с базовой таблицей

stats_df_second = stats_df_second.merge(stats_df_second_point, how='left', on=['Name','Experience'])
stats_df_second.head()


,Name,Games_Played,Goals,Assists,Points,PlusMinus_Ratings,Penalty_Minutes,Shots_on_Goal,Experience,NS_points
0,Aaron Downey,1,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0
1,Aaron Downey,3,0.0,0.0,0.0,-1.0,6.0,2.0,2,1.0
2,Aaron Downey,36,1.0,0.0,1.0,-2.0,76.0,10.0,3,2.0
3,Aaron Downey,43,1.0,1.0,2.0,1.0,69.0,14.0,4,2.0
4,Aaron Downey,37,1.0,1.0,2.0,2.0,77.0,11.0,5,NaN


In [79]:
# Дропаем NaN, с ними ничего не сделать

stats_df_second = stats_df_second[~stats_df_second['NS_points'].isna()]

In [80]:
# дробные к целым

stats_df_second[['Goals', 'Assists', 'Points', 'PlusMinus_Ratings', 'Penalty_Minutes',
         'Shots_on_Goal', 'NS_points']] = stats_df_second[['Goals', 'Assists', 'Points',
         'PlusMinus_Ratings', 'Penalty_Minutes','Shots_on_Goal', 'NS_points']].astype(int)
stats_df_second.head()

,Name,Games_Played,Goals,Assists,Points,PlusMinus_Ratings,Penalty_Minutes,Shots_on_Goal,Experience,NS_points
0,Aaron Downey,1,0,0,0,0,0,0,1,0
1,Aaron Downey,3,0,0,0,-1,6,2,2,1
2,Aaron Downey,36,1,0,1,-2,76,10,3,2
3,Aaron Downey,43,1,1,2,1,69,14,4,2
5,Aaron Downey,42,3,4,7,2,95,21,7,1


In [81]:
#факторизация имени

stats_df_second['name_factor'] = pd.factorize(stats_df_second['Name'])[0]
stats_df_second



,Name,Games_Played,Goals,Assists,Points,PlusMinus_Ratings,Penalty_Minutes,Shots_on_Goal,Experience,NS_points,name_factor
0,Aaron Downey,1,0,0,0,0,0,0,1,0,0
1,Aaron Downey,3,0,0,0,-1,6,2,2,1,0
2,Aaron Downey,36,1,0,1,-2,76,10,3,2,0
3,Aaron Downey,43,1,1,2,1,69,14,4,2,0
5,Aaron Downey,42,3,4,7,2,95,21,7,1,0
...,...,...,...,...,...,...,...,...,...,...,...
24824,Zigmund Palffy,50,22,28,50,-6,34,168,6,66,2760
24825,Zigmund Palffy,64,27,39,66,18,32,186,7,89,2760
24826,Zigmund Palffy,73,38,51,89,22,20,217,8,59,2760
24827,Zigmund Palffy,63,32,27,59,5,26,161,9,85,2760


In [82]:
features = ['Games_Played', 'Goals', 'Assists', 'Points', 'PlusMinus_Ratings', 'Penalty_Minutes', 'Shots_on_Goal', 'Experience', 'name_factor']
target = ['NS_points']

In [83]:

train, test = train_test_split(stats_df_second, test_size=0.1)

In [84]:
lin_r = LinearRegression()
lin_r.fit(train[features], train[target])

LinearRegression()

In [85]:
lin_r.coef_

array([[-1.40053501e-01,  2.33608344e-01,  2.57864484e-01,
         4.91472828e-01, -1.35754383e-02, -6.15405668e-04,
         5.72527215e-02, -3.17830710e-01,  3.18638812e-04]])

In [86]:
y_pred = lin_r.predict(test[features])

In [87]:
from sklearn.metrics import mean_squared_error

In [88]:
mse = mean_squared_error(test[target_features], y_pred)
print(f" mse = {mse}")
print(f" rmse = {np.sqrt(mse)}")

 mse = 196.64998701232983
 rmse = 14.023194607946145


In [89]:
y_pred

array([[15.90331967],
       [31.67859802],
       [11.07866528],
       ...,
       [15.49704635],
       [14.79576509],
       [40.72443922]])

In [90]:
test['pred_points'] = y_pred

In [91]:
#test[test['Name'].str.contains('Stam')]
test

,Name,Games_Played,Goals,Assists,Points,PlusMinus_Ratings,Penalty_Minutes,Shots_on_Goal,Experience,NS_points,name_factor,pred_points
3322,Bruce Driver,41,4,12,16,-1,18,62,12,37,374,15.903320
18266,Paul Gaustad,82,10,26,36,-4,85,136,6,29,2047,31.678598
6632,Denny Felsner,6,0,3,3,4,2,4,2,1,745,11.078665
15668,Max Pacioretty,66,22,18,40,-13,36,191,11,66,1757,38.245404
3873,Chris Chelios,12,0,2,2,-5,12,23,1,64,443,10.914435
...,...,...,...,...,...,...,...,...,...,...,...,...
15650,Mattias Timander,76,2,9,11,-8,24,68,5,11,1755,10.402980
8508,Georges Laraque,71,4,9,13,0,141,29,11,2,953,7.977415
4399,Claude Lapointe,29,4,8,12,5,41,40,5,9,502,15.497046
89,Adam Burish,63,8,6,14,2,91,89,5,19,11,14.795765
